In [25]:
import numpy as np
import pandas as pd
from collections import Counter
import re

In [26]:
# read in dataset
properties = pd.read_json('../data/raw/property.json').transpose()

In [27]:
# add index count
properties.reset_index(inplace=True)
properties = properties.rename(columns = {'index':'URL'})

In [28]:
# expand coordinates to x,y
properties[['x','y']] = pd.DataFrame(properties.coordinates.tolist(), index= properties.index)

In [29]:
# fill in with all missing value with nan for further process.
properties['Area'] = properties['Area'].replace( 0 , np.NaN)
properties['name']  = properties['name'].replace( "" , np.NaN)
properties['x']  =properties['x'].replace(0 , np.NaN)
properties['y']  = properties['y'].replace(0 , np.NaN) 
properties['rooms'] = properties['rooms'].where(properties['rooms'].str.len() > 0, np.nan)
properties = properties.replace(r'^\s*$', np.NaN, regex=True)


In [30]:
# inspect the number of missing values for each attribute
properties.isnull().sum()

URL               0
name              1
cost_text         1
type              1
Area           9147
coordinates       0
rooms            19
x                 2
y                 2
dtype: int64

In [31]:
# drop useless columns
# since there are too many missing values for Area variable, we decide to delete this column
properties = properties.drop(['URL', 'Area', 'coordinates'], axis=1)


In [32]:
#drop null values
properties = properties.dropna()

In [33]:
properties

,name,cost_text,type,rooms,x,y
0,56 Selwyn Street Albion VIC 3020,"$300 PW / $1,304 PCM",House,"[3 Beds, 2 Baths, 2 Parking]",-37.780782,144.813020
1,1/3 Robin Street Altona VIC 3018,$300 per week,Apartment / Unit / Flat,"[2 Beds, 1 Bath, 1 Parking]",-37.869028,144.807250
2,34 Hucker Street Ararat VIC 3377,$300 per week,House,"[3 Beds, 1 Bath, 1 Parking]",-37.276411,142.920079
3,3 Neylan Street Ararat VIC 3377,$300 per week,House,"[2 Beds, 1 Bath, 2 Parking]",-37.292905,142.921881
4,8 Suspension Street Ardeer VIC 3022,$300.00,House,"[2 Beds, 1 Bath, 2 Parking]",-37.782361,144.808833
...,...,...,...,...,...,...
9720,31 Austin Road Seaford VIC 3198,$850.00,House,"[4 Beds, 2 Baths, 2 Parking]",-38.105231,145.134350
9721,12 Morobe Street Sorrento VIC 3943,$850 pw,House,"[5 Beds, 2 Baths]",-38.354365,144.758948
9722,7/278 Kings Way South Melbourne VIC 3205,$850 per week,Townhouse,"[3 Beds, 2 Baths, 1 Parking]",-37.833083,144.966977
9723,37 Cunningham Street South Yarra VIC 3141,$850pw,House,"[3 Beds, 1 Bath, 1 Parking]",-37.840129,144.996587


In [34]:
# Check for dulipication type
Counter(properties['type'])

Counter({'House': 4558,
         'Apartment / Unit / Flat': 3839,
         'Townhouse': 1099,
         'Studio': 153,
         'Duplex': 4,
         'Villa': 22,
         'Semi-Detached': 6,
         'Terrace': 5,
         'New Apartments / Off the Plan': 3,
         'Farm': 1,
         'Acreage / Semi-Rural': 13,
         'Rural': 1})

In [35]:
# we decide to use the top five often type
# because there are too few records for other property type
keep_type = Counter(properties['type']).most_common(5)
keep_type =  [item[0] for item in keep_type]
properties = properties[properties['type'].isin(keep_type)]

In [36]:
# Check the structure of room for spliting.
Counter(properties['rooms'].str.len())

Counter({3: 7962, 2: 1709})

In [37]:
# We split room into three attributes, indicating number of beds, baths and parking
room_df = pd.DataFrame(properties['rooms'].tolist(), 
                        properties.index, columns=['Beds', 'Baths', 'Parking'])

# Only preserve numbers and set missing to zero
room_df = room_df.replace(r'\D+', '', regex=True).fillna(0)
properties = pd.concat([properties, room_df], axis=1)
properties = properties.drop(["rooms"], axis = 1)

In [38]:
# Preprocess of cost_text attribute
cost = properties[['cost_text']].astype({'cost_text':'string'})

# Since the description of cost are human typed, there are a lot of variations
# We inspect some data and decide to first recognise time scale using following
WEEK_TEXT = ['per week', 'pw', 'weekly', 'p.w', 'pw.', 'wk','/w']
MONTH_TEXT = ['per month', 'calendar month', 'pcm', '/m']

cost['cost_text'] = cost['cost_text'].str.lower().str.replace("$","").str.replace(",","")

# The below functions recognises which time scale is each record according to
# We need to separate them because different preprocessing method will be applied
def containBoth(string):
    return any(text in string for text in WEEK_TEXT) and any(text in string for text in MONTH_TEXT)
def isWeek(string):
    return any(text in string for text in WEEK_TEXT) and not any(text in string for text in MONTH_TEXT)
def isMonth(string):
    return not any(text in string for text in WEEK_TEXT) and any(text in string for text in MONTH_TEXT)
def containNone(string):
    return not any(text in string for text in WEEK_TEXT) and not any(text in string for text in MONTH_TEXT)
both = cost[cost['cost_text'].apply(containBoth)]
week = cost[cost['cost_text'].apply(isWeek)]
month = cost[cost['cost_text'].apply(isMonth)]
neither = cost[cost['cost_text'].apply(containNone)]

/tmp/ipykernel_4267/1220037653.py:9: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  cost['cost_text'] = cost['cost_text'].str.lower().str.replace("$","").str.replace(",","")


In [39]:
# For records having both week and month cost recorded, find the 3-number digits as weekly cost
# We are safe to do this because the minimum and maximum cost in dataframe is below 1000 and above 100
# For simplicity, we discard the decimals because we assume this will not affect our analysis in this scale
extractBoth = both['cost_text'].str.extract(r'(\d{3})').astype(int)
extractBoth.columns =['weekly_cost']

In [40]:
# For records containing weekly cost, we extract the three number digits as cost
# We are safe to do this because the minimum and maximum cost in dataframe is below 1000 and above 100
extractWeek = week['cost_text'].str.extract(r'(\d{3})').fillna('-1')
extractWeek = extractWeek.astype(int).replace(-1, np.nan)
extractWeek.columns =['weekly_cost']

In [41]:
# For records containing monthly cost, we extract the 3 OR 4 number digits as cost
# And convert them to weekly cost
extractMonth = month['cost_text'].apply(lambda x: re.findall(r'\d{3}\d?',x)).apply(min).to_frame().astype(int)
extractMonth = extractMonth/4
extractMonth.columns =['weekly_cost']

In [42]:
# For records not indicating weekly nor monthly cost, find the first 3-digit number as weekly cost
# because the weekly cost usually appears before monthly
extractNeither = neither['cost_text'].str.extract(r'(\d{3})').fillna('-1')
extractNeither = extractNeither.astype(int).replace(-1, np.nan)
extractNeither.columns =['weekly_cost']

In [43]:
# Combine costs we preprocessed and merge them to original dataset
costs = pd.concat([extractBoth, extractWeek, extractMonth, extractNeither])
processed_properties = pd.merge(properties, costs, left_index=True, right_index=True)
processed_properties = processed_properties.drop(['cost_text'], axis=1)

In [44]:
# Finally, get the suburb of property
processed_properties["suburb"] = processed_properties["name"].apply(lambda x: x.split()[-1])

In [45]:
processed_properties = processed_properties.rename(columns={"x": "prop_lat", "y" : "prop_long"})

In [46]:
processed_properties.to_csv('../data/curated/properties.csv', index = False) 